In [1]:
import requests

# URL del servicio Flask
url = "http://localhost:5000/generate"

# Datos a enviar
data = {
    "prompt": "Este es un ejemplo de generación de documentos.",
    "file_type": "docx"
}

try:
    # Realizar la solicitud POST
    response = requests.post(url, json=data)

    # Verificar el estado de la respuesta
    if response.status_code == 200:
        print("Solicitud exitosa!")
        # Si la respuesta incluye un archivo, puedes guardarlo
        with open("output.docx", "wb") as f:
            f.write(response.content)
        print("Archivo guardado como output.docx")
    else:
        print(f"Error {response.status_code}: {response.text}")
except requests.exceptions.RequestException as e:
    print(f"Error al conectar con la API: {e}")

Solicitud exitosa!
Archivo guardado como output.docx


In [1]:
from doc_generate import generate_and_create_file

output = 'C:/Python Projects/DocumentarIA/test/out/document.docx'
output2 = 'C:/Python Projects/DocumentarIA/test/out/document.pptx'
prompt = 'Genera un programa educatiu en català amb contingut específic de ciències per una classe de secundària. Inclou taules. El tema en questió és la cèlula. Vull 3 pàgines de contingut, amb diferents apartats i una pàgina amb exercicis o preguntes.Destaca paraules clau en color blau.'


#generate_and_create_file(prompt,file_type='docx',output=output)
generate_and_create_file(prompt,file_type='pptx',output=output2)


[H1]La Cèl·lula: Unitat Bàsica de la Vida[/H1]
[H2]Què és una cèl·lula?[/H2]
[B]La cèl·lula és l'unitat bàsica de la vida[/B]. Tots els éssers vius estan compostos per una o més cèl·lules. [COLOR:blue]Les cèl·lules són les unitats bàsiques de la vida[/COLOR], i tots els processos biològics ocorren dins elles.

[PAGEBREAK]

[H1]Estructura de la Cèl·lula[/H1]
[H2] Components de la cèl·lula [/H2]
[TABLE]
Membrana cel·lular;Citoplasma;Nuclè
Descripció;Funció;Funció
Membrana que protegeix la cèl·lula;Gelatina on es troben orgànuls|Conté material genètic
[/TABLE]
[H2]Orgànuls cel·lulars[/H2]
[B]Les cèl·lules contenen orgànuls específics[/B] que realitzen funcions concretes. [COLOR:blue]El nuclè conté el material genètic[/COLOR], mentre que el [COLOR:blue]mitocondri és responsable de la respiració cel·lular[/COLOR].

[PAGEBREAK]

[H1]Funcions Cel·lulars[/H1]
[H2] Processos cel·lulars [/H2]
[TABLE]
 Fotosíntesi; Crescuda i desenvolupament; Divisió cel·lular
 Producció de glucosa; Crecuda i evo

'C:/Python Projects/DocumentarIA/test/out/document.pptx'

In [1]:
from service_analyze.doc_analyze import extract_info_from_docs

input = ['C:/Python Projects/DocumentarIA/test/in/extract_in/pdf_largo.pdf']
output = 'C:/Python Projects/DocumentarIA/test/in/extract_out/pdf_largo.json'

prompts = ['Año del documento?', 'Resumen en 3 palabras del documento']
tipos = ['num', 'text']

extract_info_from_docs(input,output,prompts,tipos)


ModuleNotFoundError: No module named 'process_text_reader'

In [1]:
from service_edit.doc_edit import editar_doc

# Ruta de entrada y salida (cambiar según sea necesario)
input_path = "website/test/in/song.txt"  # Cambia la ruta de acuerdo a tu archivo de entrada
output_path =  "website/test/in/song.txt"   # Cambia la ruta de acuerdo a tu archivo de salida
extension = ".txt"  # Cambia según el tipo de documento
color_to_exclude = None  # Cambia según sea necesario
add_prompt = "Todo en mayúsculas."  # Cambia según sea necesario

# Llamar a la función para probar
editar_doc(input_path, output_path, extension, color_to_exclude, add_prompt)

ModuleNotFoundError: No module named 'process_text_editor'